**importing libraries**

In [ ]:
!pip install emoji
!pip install vaderSentiment

     |████████████████████████████████| 133kB 3.8MB/s 
     |████████████████████████████████| 133kB 3.8MB/s 


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.porter import PorterStemmer
import emoji
import string
import spacy
nlp = spacy.load("en_core_web_sm")
from collections import Counter

**importing data**

In [ ]:
import pandas as pd

read_file = pd.read_csv ('https://raw.githubusercontent.com/iampukar/toxic-comments-classification/master/train.csv')
toxic_data=read_file.copy()
toxic_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


**data cleaning / preprocessing**

In [ ]:
def preprocess_text(text , remove_stop=True, stem_words=False, remove_mentions_hashtags=True):

  #remove emojis
  emoji_pattern = re.compile("[" "\U0001F1E0-\U0001F6FF" "]+" , flags=re.UNICODE)
  text=emoji_pattern.sub("",text)
  text="".join([ x for x in text if x not in emoji.UNICODE_EMOJI])
  
  #remove hashtags
  if remove_mentions_hashtags:
    text =  re.sub("@(\w+)"," ",text)
    text =  re.sub("#(\w+)"," ",text)

  #remove non ASCII 
  text = re.sub("[^\x00-\x7F]+"," ",text)

  #to remove all punctuations, newline chars, tab chars etc

  regex = re.compile('[' + re.escape(string.punctuation ) + '0-9\\r\\t\\n]')
  nopunct = regex.sub(" ",text.lower())
  words = (''.join(nopunct)).split()
  
  #remove all stop words
  if (remove_stop) :
    words = [w for w in words if w not in ENGLISH_STOP_WORDS]
    words = [w for w in words if len(w) > 2]

 #to perform stemming
  if (stem_words):
    stemmer=PorterStemmer()
    words = [stemmer.stem(w) for w in words]

  return list(words)   
  

In [ ]:
toxic_data['comments_cleaned']=  toxic_data['comment_text'].apply(lambda x : ' '.join(preprocess_text(x,remove_stop=True)))

In [ ]:
toxic_data#after data cleaning

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comments_cleaned
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits username hardcore metallica ...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,aww matches background colour seemingly stuck ...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man really trying edit war just guy consta...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,make real suggestions improvement wondered sec...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,sir hero chance remember page
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,second time asking view completely contradicts...
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,ashamed horrible thing talk page
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,spitzer umm theres actual article prostitution...
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,looks like actually speedy version deleted look


**Sentiment Analysis using VADER**

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. VADER uses a combination of A sentiment lexicon is a list of lexical features (e.g., words) which are generally labeled according to their semantic orientation as either positive or negative. VADER not only tells about the Positivity and Negativity score but also tells us about how positive or negative a sentiment is.



The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between -1(most extreme negative) and +1 (most extreme positive).

positive sentiment : (compound score >= 0.05)

neutral sentiment : (compound score > -0.05) and (compound score < 0.05)

negative sentiment : (compound score <= -0.05)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment_analyser=SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    return sentiment_analyser.polarity_scores(text)['compound']

## let's get some toxic comments
toxic_comments = toxic_data.loc[toxic_data.toxic == 1]
print('toxic comment score: ', get_sentiment_score(list(toxic_comments['comment_text'])[0]))

##now for some non-toxic comment
non_toxic_comments = toxic_data.loc[((toxic_data.toxic == 0) & (toxic_data.severe_toxic==0) & (toxic_data.obscene==0) 
                                    & (toxic_data.threat == 0) & (toxic_data.insult == 0) & (toxic_data.identity_hate == 0))]
print('non toxic comment score: ', get_sentiment_score(list(non_toxic_comments['comment_text'])[0]))    

toxic comment score:  -0.7783
non toxic comment score:  0.5574


 **attempt to classify using VALOR but best results are seen with FASTTEXT.
Valor is good for sentimental analysis**

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment_analyser=SentimentIntensityAnalyzer()

def get_sentiment_score(text):
   score=sentiment_analyser.polarity_scores(text)['compound']
   
   if score >= 0.05 :
        return("Positive")
  
   elif score <= - 0.05 :
        return("Negative")
  
   else :
        return("Neutral")

In [ ]:
#checking for toxic comments 
toxic_comments=toxic_data.loc[toxic_data.toxic == 1]
print(toxic_comments.shape)
toxic_comments['sentiment_class_valor']= toxic_comments['comment_text'].apply(lambda x : get_sentiment_score(x))

print("\n\nBelow is percnetage of tweets which are correcly classified by VALOR:")
print((toxic_comments['sentiment_class_valor'].value_counts()['Negative'] / toxic_comments['sentiment_class_valor'].shape[0])*100)

(15294, 9)


Below is percnetage of tweets which are correcly classified by VALOR:
73.55171962861253


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
#checking for non toxic comments 
non_toxic_comments = toxic_data.loc[((toxic_data.toxic == 0) & (toxic_data.severe_toxic==0) & (toxic_data.obscene==0) 
                                    & (toxic_data.threat == 0) & (toxic_data.insult == 0) & (toxic_data.identity_hate == 0))]

print(non_toxic_comments.shape)
non_toxic_comments['sentiment_class_valor']= non_toxic_comments['comment_text'].apply(lambda x : get_sentiment_score(x))

print("\n\nBelow is percnetage of tweets which are correcly classified by VALOR:")
print((non_toxic_comments['sentiment_class_valor'].value_counts()['Positive'] / non_toxic_comments['sentiment_class_valor'].shape[0])*100)

(143346, 9)


Below is percnetage of tweets which are correcly classified by VALOR:
52.934856919621055


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


**FastText for Text Classification**

we are going to study FastText which is another extremely useful module for word embedding and text classification. FastText has been developed by Facebook and has shown excellent results on many NLP problems, such as semantic similarity detection and text classification.



In [ ]:
toxic_data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comments_cleaned
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits username hardcore metallica ...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,aww matches background colour seemingly stuck ...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man really trying edit war just guy consta...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,make real suggestions improvement wondered sec...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,sir hero chance remember page
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,congratulations use tools talk
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,cocksucker piss work
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,vandalism matt shirvington article reverted do...
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,sorry word nonsense offensive intending write ...
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,alignment subject contrary dulithgow


In [ ]:
# ignore severe_toxic, obscene, threat, insult, identity_hate
toxic_non_toxic_data = toxic_data.loc[((toxic_data.severe_toxic==0) & (toxic_data.obscene==0) 
                                    & (toxic_data.threat == 0) & (toxic_data.insult == 0) 
                                       & (toxic_data.identity_hate == 0))]


In [ ]:
toxic_non_toxic_data['toxic'].value_counts()

#preparing target varibales
print(type(toxic_non_toxic_data[['toxic']])) #this to be target
print(type(toxic_non_toxic_data['toxic'])) #not this

y=toxic_non_toxic_data[['toxic']]

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [ ]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(toxic_non_toxic_data,y,test_size=0.2)
print("size of X_train,y_train",X_train.shape,y_train.shape)
print("size of X_test,y_test",X_test.shape,y_test.shape)

size of X_train,y_train (119209, 9) (119209, 1)
size of X_test,y_test (29803, 9) (29803, 1)


In [ ]:
#Before we train FastText models to perform text classification, it is pertinent to mention that FastText accepts data in a special format, which is as follows:

#_label_tag This is sentence 1
#_label_tag2 This is sentence 2.
X_train['toxic']=  ['__label__'+str(s) for s in X_train['toxic']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
import csv
data=X_train[['comment_text','toxic']]
data.to_csv('comments.txt', index=False, sep=' ', header=False,quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [ ]:
data

,comment_text,toxic
37381,REDIRECT Talk:Riffle shuffle permutation,__label__0
16987,:Image:IsleRoyalePlane.jpg\nDear MetsBot:\n\nW...,__label__0
93313,"Willmcw: Okay, but I don't undersatnd what you...",__label__0
136701,Personal life\nNick has never confirmed this '...,__label__0
137066,"""\n\nMore nonsense. Your rationalization migh...",__label__0
...,...,...
69720,"""\n\nYou are copying one-sided (Israeli) mater...",__label__0
59064,"As I see it, you aren't reading my sources. I...",__label__0
146860,"""\nPlease stop. If you continue to vandalize p...",__label__0
126747,"""\n\nSerbian casualties again\n\nJust one link...",__label__0


In [ ]:
#training for 4 epochs
!pip install fasttext
import fasttext
model=fasttext.train_supervised(input='comments.txt',epoch=4)
print(model)

     |████████████████████████████████| 71kB 2.6MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3094687 sha256=0b17f50ab865a0033020d0d116831d71e251e325243fc2a36c71e59955929454
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
#predicting for one example, 
text='Touching on this, I found a useless remark that appeared to be vandalism. Pertaining an opinion on how seriously the college takes its football, its now been rectified'
op=model.predict(text)
print(op)
print("label predicted is ",op[0][0].split('__label__')[1])

(('__label__0',), array([0.97349387]))
label predicted is  0


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#extracting prediction out of the fasttext output
def get_prediction(fast_out):
   return (fast_out[0][0].split('__label__')[1])

In [ ]:
#to find training accuracy
pred =[get_prediction(model.predict(' '.join(preprocess_text(x)))) for x in list(X_train['comment_text'])]

pred_int=[int(i) for i in pred]
print(pred_int)
print(y_train.values.reshape(1,y_train.values.shape[0]).tolist()[0])

print('\naccuracy:',accuracy_score(pred_int,y_train.values),'\n')
print('\nF1-score:',f1_score(pred_int,y_train),'\n')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
#to find testing accuracy
pred =[get_prediction(model.predict(' '.join(preprocess_text(x)))) for x in list(X_test['comment_text'])]

pred_int=[int(i) for i in pred]
print(pred_int)
print(y_test.values.reshape(1,y_test.values.shape[0]).tolist()[0])

print('\naccuracy:',accuracy_score(pred_int,y_test.values),'\n')
print('\nF1-score:',f1_score(pred_int,y_test),'\n')

#accuracy obatined is better than VADER.

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 